In [3]:
import uuid

from essential_generators import DocumentGenerator
from tqdm import tqdm

import chromadb

client = chromadb.HttpClient()

col = client.get_or_create_collection("test1")
# col.add(ids=["0"],documents=["test doc"],metadatas=[{"a":"1","b":2,"c":True}])
batch_size = 100
batches= 1000000
gen = DocumentGenerator()
for batch in tqdm(range(batches)):
    batch_start = batch*batch_size
    batch_end = batch_start+batch_size
    col.add(ids=[f"{uuid.uuid4()}" for _id in range(batch_start,batch_end)],documents=[gen.sentence() for _ in range(batch_start,batch_end)],metadatas=[{"a":f"id-{_id}","b":_id,"c":True} for _id in range(batch_start,batch_end)])

# print(col.get(where={"$and":[{"$or":[{"a":"1"},{"b":2}]},{"c":True}]}))

  0%|          | 2456/1000000 [4:18:19<1748:40:13,  6.31s/it]  


KeyboardInterrupt: 

In [1]:
from concurrent.futures import ThreadPoolExecutor

batch_size = 100
batches = 1000000
gen = DocumentGenerator()

def process_batch(batch):
    batch_start = batch * batch_size
    batch_end = batch_start + batch_size
    col.add(
        ids=[f"{uuid.uuid4()}" for _ in range(batch_start, batch_end)],
        documents=[gen.sentence() for _ in range(batch_start, batch_end)],
        metadatas=[{"a":f"id-{_id}", "b":_id, "c":True} for _id in range(batch_start, batch_end)]
    )
num_threads = 5  # Adjust this value as needed
with ThreadPoolExecutor() as executor:
    list(tqdm(executor.map(process_batch, range(batches)), total=batches))

NameError: name 'DocumentGenerator' is not defined

In [10]:

print(col.get(where={"b":{"$lte":7}}))

{'ids': ['0', '9b72d30d-8441-49ec-b159-efcbf862f4fb', '55041953-f424-400a-bfad-3f516bea4965', '3014ec4c-7a86-497d-965a-92ff557cf853', '2ff6983c-a2ab-4f53-9f27-fa01fa85bbf1', '9a101542-4ee9-4409-9363-5a9a9a0c56b3', '93647f99-daea-4d78-afbb-1dcef8415139', '5e688377-602c-404c-8d79-6a1557fbbd1a', 'b8e80537-f761-4601-8a1d-138c09f4e53c', '257e1090-381a-4db8-a8d3-51111a37903a', 'c3e0bf10-599d-451e-958c-444914d3ee38', 'a2645c75-53cf-452b-a661-03a0a18c5d59', 'f00d905a-1c31-4fc3-96af-e283ee0c573f', '2f34b45c-270e-4d30-96bf-9e45fe6a8315', '25a8b450-2c6e-42de-abdc-da7bddf7d96b', '08e310ae-7380-4991-b153-5d15af81fec8', 'feed95d9-2868-4879-861b-24ddd5d705a3', '020a529a-4b5f-43d1-b38e-b106b885c04a', '8af9cffc-4914-4a65-81dd-f582987519d5', 'a4e21b4c-b7c9-42ad-8fef-d0cd8a073dff', 'f8146f6c-2a9d-433d-b6b2-db654f03cfbc', 'd2131ef8-de9a-4b8b-9215-37ca4f4bb9dc', '9a014b6c-9ecf-44f2-9366-2fed18dec294', 'c73b1288-7421-4674-93fb-138bf4e16147', 'd7496956-20cc-4a58-801c-07c75ac29962', 'f9958bca-0132-4ac5-8746-f

In [91]:
import uuid
import chromadb

client = chromadb.HttpClient()
col = client.get_or_create_collection("test1")
print(len(col.query(where={"b":{"$gt":245500}},query_texts=["ocean is blue"])))

5


In [4]:
from chromadb.db.base import ParameterValue, get_sql
from chromadb.api.types import SqlBackedIndex, IndexType
from chromadb.db.utils import IndexQuery
from pypika import Table

# TEsting SQL queries
index = SqlBackedIndex(name="test_index", columns={'int_value', 'float_value'},
                       index_type=IndexType.METADATA)
collection_id="2ff6983c-a2ab-4f53-9f27-fa01fa85bbf1"
embedding_metadata = Table('embedding_metadata')
q = (IndexQuery()
     .create_index(f'cust_idx_{index["name"]}')
     .on(embedding_metadata)
     # , 'segment_id', 'key', 'int_value', 'float_value'
     .columns(embedding_metadata.key, embedding_metadata.segment_id,
              *(embedding_metadata[col] for col in index["columns"]))
     .where(embedding_metadata.key != "chroma:document")
     .where(embedding_metadata.segment_id == ParameterValue(collection_id))
     )
print(get_sql(q))

('CREATE INDEX cust_idx_test_index ON "embedding_metadata"(key, segment_id, float_value, int_value) WHERE "key"<>\'chroma:document\' AND "segment_id"=?', ('2ff6983c-a2ab-4f53-9f27-fa01fa85bbf1',))


In [1]:
from chromadb import Settings
from chromadb.api.types import  SqlBackedIndex, IndexType
import chromadb
client = chromadb.PersistentClient(path="./metadata_index_testing", settings=Settings(allow_reset=True))
client.reset()
col = client.get_or_create_collection("indexed_col")
print(col)
col.add(ids=["0"],documents=["test doc"],metadatas=[{"a":"1","b":2,"c":True}])

index = SqlBackedIndex(name="test_index", columns={'int_value', 'float_value'},
                       index_type=IndexType.METADATA)
client._create_collection_index(collection_id=col.id, indices=[index])

print(col.get(where={"b":{"$lte":7}}))




name='indexed_col' id=UUID('a18260d9-2555-4237-b70a-61c0a2bbedfa') metadata=None
CREATE INDEX cust_idx_test_index ON "embedding_metadata"(key, segment_id, int_value, float_value) WHERE "key"<>'chroma:document' AND "segment_id"='10e20e3b-b3bd-425a-acc4-2cb8b212d46e'
SQL:  SELECT "embeddings"."id","embeddings"."embedding_id","embeddings"."seq_id","embedding_metadata"."key","embedding_metadata"."string_value","embedding_metadata"."int_value","embedding_metadata"."float_value","embedding_metadata"."bool_value" FROM "embeddings" LEFT JOIN "embedding_metadata" ON "embeddings"."id"="embedding_metadata"."id" WHERE "embeddings"."segment_id"=? AND "embeddings"."id" IN (SELECT "id" FROM "embedding_metadata" WHERE "segment_id"=? AND "key"=? AND "key"<>'chroma:document' AND ("int_value"<=? OR "float_value"<=?)) ORDER BY "embeddings"."id"
Execution time: 0.0003 seconds
{'ids': ['0'], 'embeddings': None, 'metadatas': [{'a': '1', 'b': 2, 'c': True}], 'documents': ['test doc']}


In [2]:
client._drop_collection_index(collection_id=col.id, index_names=[index['name']])

DROP INDEX "cust_idx_test_index"
